In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
df = pd.DataFrame()
df = pd.read_csv('/Users/arzoo/Desktop/SEM1/Intro to DL/sentiment-data/train.csv', encoding = "utf-8", 
                 names=["Sentiment", "Text"])
df['Text'] = df['Text'].astype(str) 
df['positive'] = (df.Sentiment == "postive")
df['negative'] = (df.Sentiment == "negative")
df['positive'] = df['positive'].astype(int)
df['negative'] = df['negative'].astype(int)

In [3]:

X_train = df['Text'].values 

y_train = df[['positive','negative']].values



In [4]:
dff = pd.DataFrame()
dff = pd.read_csv('/Users/arzoo/Desktop/SEM1/Intro to DL/sentiment-data/test.csv', encoding = "utf-8", 
                 names=["Sentiment", "Text"])
dff['Text'] = dff['Text'].astype(str) 
X_test = dff['Text'].values
x_test = X_test
dff['positive'] = (dff.Sentiment == "postive")
dff['negative'] = (dff.Sentiment == "negative")
dff['positive'] = dff['positive'].astype(int)
dff['negative'] = dff['negative'].astype(int)
y_test = dff[['positive','negative']].values


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

X_test = tokenizer.texts_to_sequences(X_test)
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 60

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [6]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
#commas have been removed from the word-vectors.txt file
glove_file = open('/Users/arzoo/Desktop/SEM1/Intro to DL/sentiment-dataa/word-vectors.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
    
glove_file.close()



In [7]:
embedding_matrix = zeros((vocab_size, 50))

for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        
print(embedding_matrix)


[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.41800001  0.24968    -0.41242    ... -0.18411    -0.11514
  -0.78580999]
 [ 0.26818001  0.14346001 -0.27877    ... -0.63209999 -0.25027999
  -0.38097   ]
 ...
 [ 0.70876002 -0.63502002  0.6243     ...  0.55676001  0.21197
  -0.36195999]
 [ 0.058441    0.43887001 -1.31029999 ... -0.64341998 -0.34671
   0.90706998]
 [-0.24207    -0.73237002  0.23311999 ... -0.23586001 -0.078379
  -0.20638999]]


In [13]:
from keras.layers.core import Dense, Dropout
from keras.layers import LSTM, GRU, SimpleRNN
def test(mod):
    model = Sequential()
    embedding_layer = Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=maxlen , trainable=False)
    model.add(embedding_layer)
    model.add(mod)
    model.add(Dense(60, activation='tanh'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    print(model.summary())
    history = model.fit(X_train, y_train, batch_size=500, epochs=1, verbose=1)
    score = model.evaluate(X_test, y_test, verbose=1)
    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])
    return


In [14]:
test(SimpleRNN(64))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 60, 50)            2042650   
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 64)                7360      
_________________________________________________________________
dense_7 (Dense)              (None, 60)                3900      
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 122       
Total params: 2,054,032
Trainable params: 11,382
Non-trainable params: 2,042,650
_________________________________________________________________
None
Epoch 1/1
5000/5000 [==============================] - 6s 1ms/step
Test Score: 0.6892178791999817
Test Accuracy: 0.5414


In [15]:
test(LSTM(64))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 60, 50)            2042650   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_9 (Dense)              (None, 60)                3900      
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 122       
Total params: 2,076,112
Trainable params: 33,462
Non-trainable params: 2,042,650
_________________________________________________________________
None
Epoch 1/1
5000/5000 [==============================] - 7s 1ms/step
Test Score: 0.6035113176345825
Test Accuracy: 0.6786


In [16]:
test(GRU(64))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 60, 50)            2042650   
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                22080     
_________________________________________________________________
dense_11 (Dense)             (None, 60)                3900      
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 122       
Total params: 2,068,752
Trainable params: 26,102
Non-trainable params: 2,042,650
_________________________________________________________________
None
Epoch 1/1
5000/5000 [==============================] - 6s 1ms/step
Test Score: 0.6685330257415771
Test Accuracy: 0.5922
